In [1]:
# 全局设置
import os
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

基于 SQL 数据库构建的因子库的对应关系：整个数据库对应于因子库, 每张数据库表对应于因子表, 每张数据库表的字段对应于单个因子. 本质上是将一个二维的因子数据矩阵挤压成具有二重索引的一维向量. 对于因子数据的访问, 内部使用标准的 SQL 查询语句完成. 关系型数据库是一种相对成熟的数据组织方式, 在完整性、一致性以及操作的便利上都有着充分的优势.

![SQL因子库](../images/SQL因子库.png)

![UML-SQLDB](../images/UML-SQLDB.png)

# QSSQLObject

QSSQLObject 参数集:
* 数据库类型: str, 可选 "MySQL"(默认值), "SQL Server", "Oracle"
* 数据库名: str, 默认值 "Scorpion"
* IP地址: str, 默认值, "127.0.0.1"
* 端口: int, 默认值 3306
* 用户名: str, 默认值 "root"
* 密码: str, 默认值 ""
* 表名前缀: str, 默认值 ""
* 字符集: str, 可选 "utf8"(默认值), "gbk", "gb2312", "gb18030", "cp936", "big5"
* 连接器: str, 可选 "default"(默认值), "cx_Oracle", "pymssql", "mysql.connector", "pymysql", "pyodbc". 如果选择 "default", 对于 MySQL 实际为 "mysql.connector" 或 "pyodbc"; 对于 SQL Server 实际为 "pymssql" 或 "pyodbc"; 对于 Oracle 实际为 "cx_Oracle" 或 "pyodbc".
* 数据源: str, 默认值 "", 当连接器为 "pyodbc" 时, 如果创建了 ODBC 数据源, 可以将数据源名赋予该参数
* 调整表名: bool, 默认值 False

QSSQLite3Object 继承自 QSSQLObject, 参数集:
* 数据库类型: str, 固定值 "sqlite3"
* 连接器: str, 固定值 "sqlite3"
* sqlite3文件: str, 可选 ":memory:"(默认值), 文件路径, ":memory:" 表示内存数据库

QSClickHouseObject 继承自 QSSQLObject, 参数集:
* 数据库类型: str, 固定值 "ClickHouse"
* 连接器: str, 可选 "default"(默认值), "clickhouse-driver". 如果选择 "default", 实际为 "clickhouse-driver". 

# 外部因子库

## JYDB

继承自 QSSQLObject, FactorDB, 参数集:
* 名称: str, 默认值 "JYDB"
* 库信息文件: 文件路径, 默认值 QuantStudio 包目录下 Resource 目录下的 JYDBInfo.xlsx, 记录了聚源数据库相关配置信息

In [6]:
# 示例因子库 JYDB
JYDB = QS.FactorDB.JYDB()
JYDB.connect();

### 获取时点序列

#### 获取交易日序列

获取交易日序列 getTradeDay, 输入参数:
* start_date: date 或者 datetime, 默认值 None, 待获取的交易日序列的起始日, None 表示最早的第一个交易日
* end_date: date 或者 datetime, 默认值 None, 待获取的交易日序列的结束日, None 表示今天
* exchange: str, 可选 "SSE"(默认值), "SZSE", "SHFE", "DCE", "CZCE", "INE", "CFFEX", 指定的交易所
* output_type: str, 可选 "date"(默认值), "datetime", 返回的交易日数据类型

In [7]:
# 获取交易日序列
print(JYDB.getTradeDay(start_date=dt.datetime(2022,1,1), end_date=dt.datetime(2022,1,20), exchange="SSE", output_type="datetime"))

[datetime.datetime(2022, 1, 4, 0, 0), datetime.datetime(2022, 1, 5, 0, 0), datetime.datetime(2022, 1, 6, 0, 0), datetime.datetime(2022, 1, 7, 0, 0), datetime.datetime(2022, 1, 10, 0, 0), datetime.datetime(2022, 1, 11, 0, 0), datetime.datetime(2022, 1, 12, 0, 0), datetime.datetime(2022, 1, 13, 0, 0), datetime.datetime(2022, 1, 14, 0, 0), datetime.datetime(2022, 1, 17, 0, 0), datetime.datetime(2022, 1, 18, 0, 0), datetime.datetime(2022, 1, 19, 0, 0), datetime.datetime(2022, 1, 20, 0, 0)]


### 获取ID序列

#### 股票 ID

获取股票 ID 序列 getStockID, 输入参数:
* exchange: str 或者 tuple(str), 默认值 ("SSE", "SZSE", "BSE"). 指定的交易所或者交易所列表, 可选的交易所有 "SSE", "SZSE", "BSE", "HKEX";
* date: None(默认值) 或者 datetime, 指定日, None 表示今天;
* is_current: bool, 默认值 True. False 表示获取上市日期在指定日之前的股票, True 表示上市日期在指定日之前且当前尚未退市的股票;
* start_date: None(默认值) 或者 datetime. 起始日, 如果非 None, is_current=False 表示提取在 start_date 至 date 之间上市过的股票 ID, is_current=True 表示提取在 start_date 至 date 之间均保持上市的股票. None 表示足够早的日期.

In [ ]:
# 获取股票 ID 序列
print(JYDB.getStockID(exchange=("SSE", "SZSE", "BSE"), date=dt.datetime(2022,1,1), is_current=True, start_date=None)[:10])

#### 公募基金 ID

获取公募基金 ID 序列 getMutualFundID, 输入参数:
* date: None(默认值) 或者 datetime, 指定日, None 表示今天
* is_current: bool, 默认值 True, False 表示成立日在指定日之前的基金, True 表示成立日在指定日之前且尚未清盘的基金
* start_date: None(默认值) 或者 datetime, 起始日, 如果非 None, is_current=False 表示提取在 start_date 至 date 之间存续过的公募基金 ID, is_current=True 表示提取在 start_date 至 date 之间均保持存续的公募基金. None 表示足够早的日期.

In [ ]:
# 获取公募基金 ID 序列
print(JYDB.getMutualFundID(date=dt.datetime(2022,1,1), is_current=True, start_date=None)[:10])

#### 债券 ID

获取债券 ID 序列 getBondID, 输入参数:
* exchange: None(默认值), str 或者 tuple(str), 指定的交易所或者交易所列表, None 表示获取库里所有的债券交易所;
* date: None(默认值) 或者 datetime, 指定日, None 表示今天
* is_current: bool, 默认值 True, False 表示存续起始日在指定日之前的债券, True 表示存续起始日在指定日之前且尚未到期的债券
* start_date: None(默认值) 或者 datetime, 起始日, 如果非 None, is_current=False 表示提取在 start_date 至 date 之间存续过的债券 ID, is_current=True 表示提取在 start_date 至 date 之间均保持存续的债券. None 表示足够早的日期.

In [ ]:
# 获取债券 ID 序列
print(JYDB.getBondID(exchange=None, date=dt.datetime(2022,1,1), is_current=True, start_date=None)[:10])

#### 期货 ID

获取期货 ID 序列 getFutureID, 输入参数:
* exchange: None, str 或者 tuple(str), 默认值 "CFFEX", 指定的交易所或者交易所列表, 可选的交易所有 "CFFEX", "DCE", "CZCE", "SHFE", "INE";
* future_code: None(默认值), str 或者 tuple(str), 期货代码或者期货代码列表, None 表示所有期货代码
* date: None(默认值) 或者 datetime, 指定日, None 表示今天
* is_current: bool, 默认值 True. False 表示上市日在指定日之前的期货, True 表示上市日在指定日之前且尚未退市的期货
* start_date: None(默认值) 或者 datetime, 起始日, 如果非 None, is_current=False 表示提取在 start_date 至 date 之间上市过的期货 ID, is_current=True 表示提取在 start_date 至 date 之间均保持上市的期货. None 表示足够早的日期.
* contract_type: str, 可选 "月合约"(默认值), "连续合约", "所有", 合约类型

In [ ]:
# 获取期货 ID 序列
print(JYDB.getFutureID(exchange="CFFEX", future_code="IF", date=dt.datetime(2022,1,1), is_current=True, start_date=None, contract_type="月合约")[:10])

获取期货代码序列 getFutureCode, 输入参数:
* exchange: str 或者 tuple(str), 默认值 ("SHFE", "INE", "DCE", "CZCE", "CFFEX"), 指定的交易所或者交易所列表, 可选的交易所有 "CFFEX", "DCE", "CZCE", "SHFE", "INE";
* date: None(默认值) 或者 datetime, 指定日, None 表示今天
* is_current: bool, 默认值 True. False 表示上市日在指定日之前的期货d代码, True 表示上市日在指定日之前且尚未退市的期货代码

In [ ]:
# 获取期货代码序列
print(JYDB.getFutureCode(exchange=("SHFE", "INE", "DCE", "CZCE", "CFFEX"), date=None, is_current=True))

#### 期权 ID

获取期权 ID 序列 getOptionID, 输入参数:
* option_code: str, 默认值 "510050", 期权代码
* date: None(默认值) 或者 datetime, 指定日, None 表示今天;
* is_current: bool, 默认值 True. False 表示获取上市日期在指定日之前的期权, True 表示上市日期在指定日之前且当前尚未退市的期权;
* start_date: None(默认值) 或者 datetime. 起始日, 如果非 None, is_current=False 表示提取在 start_date 至 date 之间上市过的期权 ID, is_current=True 表示提取在 start_date 至 date 之间均保持上市的期权. None 表示足够早的日期.

In [ ]:
# 获取期权 ID 序列
print(JYDB.getOptionID(option_code="510050", date=None, is_current=True, start_date=None)[:10])

#### 行业 ID

获取行业 ID 序列 getIndustryID, 输入参数:
* standard: str, 默认值 "中信行业分类", 行业分类标准
* level: int 默认值 1, 行业分类级别
* date: None(默认值) 或者 datetime, 指定日, None 表示今天;
* is_current: bool, 默认值 True. False 表示获取生效日期在指定日之前的行业, True 表示生效日期在指定日之前且当前尚未失效的行业;
* start_date: None(默认值) 或者 datetime. 起始日, 如果非 None, is_current=False 表示提取在 start_date 至 date 之间起效过的行业 ID, is_current=True 表示提取在 start_date 至 date 之间均保持有效的行业. None 表示足够早的日期.

In [ ]:
# 获取行业 ID 序列
print(JYDB.getIndustryID(standard="中信行业分类", level=1, date=None, is_current=True, start_date=None))

## WindDB2

# 本地因子库

## SQLDB

继承自 QSSQLObject, WritableFactorDB, 初始化方法 `__init__`:
* sys_args: dict, 因子库参数集
* config_file: None 或者 str, 因子库的配置文件地址, None 表示使用默认配置文件, 默认文件名为 "SQLDBConfig.json", 默认路径为参见: [配置文件](../基本框架.ipynb#QuantStudio-对象)

参数集:
* 名称: str, 默认值 "SQLDB"
* 检查写入值: bool, 默认值 False. 是否检查写入的数据是否为 list 类型, 对于 list 类型的数据展开后写入
* 忽略字段: list(str), 默认值 [], 数据库表中需要忽略不进行识别作为数据项的字段列表
* 内部前缀: str, 默认值 "qs_", 识别为因子表的数据库表所必需的表名前缀
* 因子表参数: dict, 默认值 {}, 通过 getTable 创建因子表时传递给它的默认参数集
* 时点字段: str, 默认值 "datetime", 因子表默认的时点字段, 通过 getTable 创建因子表时传递给它
* ID字段: str, 默认值 "code", 因子表默认的 ID 字段, 通过 getTable 创建因子表时传递给它
* 检查缺失容许: bool, 默认值 False, 写入数据时是否检查相关的字段是否有非空约束, 如果某字段有非空约束, 则将其相应的数据去除缺失后再写入(同时给出 warning)

## ClickHouseDB

继承自 QSClickHouseObject, SQLDB, 参数集:
* 名称: str, 默认值 "ClickHouseDB"
* 时点字段: str, 默认值 "qs_datetime", 因子表默认的时点字段, 通过 getTable 创建因子表时传递给它
* ID字段: str, 默认值 "qs_code", 因子表默认的 ID 字段, 通过 getTable 创建因子表时传递给它

## SQLite3DB

继承自 QSSQLObject, SQLDB, 参数集:
* 名称: str, 默认值 "SQLite3DB"
* 时点格式: str, 默认值 "%Y-%m-%d", 因为 sqlite3 数据库没有日期和时间数据类型, 使用 text 代替, 所以需要设定时点字符串格式

In [3]:
# 示例因子库 SQLite3DB
import QuantStudio.FactorDataBase.SQLite3DB
# SQLite3DB = QuantStudio.FactorDataBase.SQLite3DB.SQLite3DB(sys_args={"sqlite3文件": "../Data/SQLite3/TestData.sqlite3", 
#                                                                      "内部前缀": "", 
#                                                                      "时点字段": "datetime",
#                                                                      "ID字段": "code",
#                                                                      "时点格式": "%Y-%m-%d",
#                                                                      "因子表参数": {"忽略时间": True}}).connect();
SQLite3DB = QuantStudio.FactorDataBase.SQLite3DB.SQLite3DB(config_file="../Config/SQLite3DBConfig.json").connect();

# 因子表

SQL_Table 参数集:
* 筛选条件: str, 默认值 "", 形成 SQL 查询时附加到 WHERE 子句上的条件. 比如 "({Table}.field1>10) AND ({Table}.field2 IN ('a','b')", 其中 {Table} 会自动替换为相应的数据库表名
* 因子表类型: str, 固定值, 可选 "WideTable", "NarrowTable", "FeatureTable", "TimeSeriesTable", "MappingTable", "ConstituentTable", "FinancialTable" 等. 只能在 getTable 时传入，因子表创建后不可改变, 用于指明形成的因子表的类型.
* 预筛选ID: bool, 默认值 True. 是否在 SQL 查询中筛选 ID, 如果为 True, 则在形成的 SQL 查询中的 WHERE 子句中会有 {Table}.ID字段 IN (...) 条件, 否则为 {Table}.ID字段 IS NOT NULL. 如果提取数据的 ID 不多，建议为 True.
* 时点字段: str, 默认内部自动判断. 因子表用于表示时点维度的字段名
* ID字段: str, 默认内部自动判断. 因子表用于表示 ID 维度的字段名
* 使用索引: list(str), 如果非空, 则在 SQL 查询中附加子句 USE INDEX (索引1, 索引2, ...)
* 强制索引: list(str), 如果非空, 则在 SQL 查询中附加子句 FORCE INDEX (索引1, 索引2, ...)
* 忽略索引: list(str), 如果非空, 则在 SQL 查询中附加子句 IGNORE INDEX (索引1, 索引2, ...)

In [4]:
# SQL_Table 原始数据
SQLStr = "SELECT * FROM test_WideTable1"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(9)]
IDs = ["000001.SZ"]

Args = {
    "因子表类型": "WideTable",
    "筛选条件": "",#"{Table}.factor1 IS NOT NULL",
    "预筛选ID": True,
    "时点字段": "datetime",
    "ID字段": "code",
    "使用索引": [],
    "强制索引": [],
    "忽略索引": [],
}
FT = SQLite3DB.getTable("test_WideTable1", args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=IDs, dts=DTs).iloc[:, :, 0])

库原始数据 : 
     datetime       code  factor1  factor2
0  2020-01-01  000001.SZ      1.0      NaN
1  2020-01-02  000001.SZ      NaN      2.0
2  2020-01-03  000001.SZ      3.0      4.0
3  2020-01-04  000001.SZ      NaN      NaN
4  2020-01-05  000001.SZ      NaN      6.0
5  2020-01-06  000001.SZ      5.0      7.0
6  2020-01-07  000001.SZ      NaN      NaN
7  2020-01-08  000001.SZ      8.0      9.0
参数 : 
{'因子表类型': 'WideTable', '筛选条件': '', '预筛选ID': True, '时点字段': 'datetime', 'ID字段': 'code', '使用索引': [], '强制索引': [], '忽略索引': []}
数据 : 
            factor1  factor2
2020-01-01      1.0      NaN
2020-01-02      NaN      2.0
2020-01-03      3.0      4.0
2020-01-04      NaN      NaN
2020-01-05      NaN      6.0
2020-01-06      5.0      7.0
2020-01-07      NaN      NaN
2020-01-08      8.0      9.0
2020-01-09      NaN      NaN


## WideTable

继承自 SQL_Table, 参数集:
* 缺失填充相关参数:
    + 回溯天数: float(可取 inf), 默认值 0, 缺失填充回溯的天数
    + 只起始日回溯: bool, 默认值 False, 如果为 True, 表示只对提取数据的第一个时点进行缺失填充, 之后的时点不填充
    + 只回溯非目标日: bool, 默认值 False, 如果为 True, 表示只用不在提取时点序列中的数据进行缺失填充
    + 只回溯时点: bool, 默认值 False, 如果为 True, 表示所有因子统一沿着时点字段进行回溯填充, 不单独填充
* 公告时点字段: str, 默认值自动判断. 用作公告时点的字段名, 如果非 None, 表示考虑数据的公布时点, 即某个时点所能获取的数据必须保证其在公告时点和截止时点之后
* 忽略时间: bool, 默认值 False, 时点字段和公告时点字段是否将其时间调整为 00:00:00
* 截止日期递增: bool, 默认值 False
* 多重映射相关参数:
    + 多重映射: bool, 默认值 False, 是否为高维数据, 即时点和 ID 两个维度无法唯一索引单个数据, 默认形成的数据在单个时点单个 ID 处以 list 形式表达
    + 排序字段: list(tuple), 默认值 [], 对提取出的数据进行排序的设置, 比如 [("factor1", "ASC"), ("factor2", "DESC")] 表示提取出的数据根据时点和 ID 分组后先按照 factor1 升序排列再按照 factor2 降序排列
    + 算子: function, 默认值 lambda x: x.tolist(), 对于单个时点单个 ID 处的数据 apply 的函数(f(x), 其中 s 为 Series)
    + 算子数据类型: str, 可选 "object"(默认值), "double", "string", 上述算子的输出值的数据类型
    + 附加字段: list(str), 默认值 [], 传递个上述算子的额外字段数据, 如果非空, 则上述算子为 f(x), 其中 x 为 DataFrame, 其中第一个 column 为目标因子, 其余 column 为附加字段
* 回溯期相关参数:
    + 回溯期数: None 或者 int, 默认值 None, 进行数据回溯的偏移量
    + 原始值回溯天数: float(可取 inf), 默认值 0, 提取原始数据时的回溯天数

数据库数据示例：
![Wide_Table_Demo](../images/Wide_Table_Demo.png)

### 缺失填充

In [4]:
# WideTable 原始数据
Args = {
    "因子表类型": "WideTable", 
    "回溯天数": 0, 
    "只起始日回溯": False, 
    "只回溯非目标日": False, 
    "只回溯时点": False
}
FT = SQLite3DB.getTable("test_WideTable1", args=Args)
DTs = FT.getDateTime()
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=["000001.SZ"], dts=DTs).iloc[:, :, 0])

参数 : 
{'因子表类型': 'WideTable', '回溯天数': 0, '只起始日回溯': False, '只回溯非目标日': False, '只回溯时点': False}
数据 : 
            factor1  factor2
2020-01-01      1.0      NaN
2020-01-02      NaN      2.0
2020-01-03      3.0      4.0
2020-01-04      NaN      NaN
2020-01-05      NaN      6.0
2020-01-06      5.0      7.0
2020-01-07      NaN      NaN
2020-01-08      8.0      9.0


In [5]:
# WideTable 回溯天数
Args = {
    "因子表类型": "WideTable", 
    "回溯天数": 1, # 0, 1, 2, np.inf
    "只起始日回溯": False, 
    "只回溯非目标日": False, 
    "只回溯时点": False
}
FT = SQLite3DB.getTable("test_WideTable1", args=Args)
DTs = [dt.datetime(2020,1,4)+dt.timedelta(i) for i in range(6)]
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=["000001.SZ"], dts=DTs).iloc[:, :, 0])

参数 : 
{'因子表类型': 'WideTable', '回溯天数': 1, '只起始日回溯': False, '只回溯非目标日': False, '只回溯时点': False}
数据 : 
            factor1  factor2
2020-01-04      3.0      4.0
2020-01-05      NaN      6.0
2020-01-06      5.0      7.0
2020-01-07      5.0      7.0
2020-01-08      8.0      9.0
2020-01-09      8.0      9.0


In [6]:
# WideTable 只起始日回溯
Args = {
    "因子表类型": "WideTable", 
    "回溯天数": 1, # 0, 1, 2, np.inf
    "只起始日回溯": True, 
    "只回溯非目标日": False, 
    "只回溯时点": False
}
FT = SQLite3DB.getTable("test_WideTable1", args=Args)
DTs = [dt.datetime(2020,1,4)+dt.timedelta(i) for i in range(6)]
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=["000001.SZ"], dts=DTs).iloc[:, :, 0])

参数 : 
{'因子表类型': 'WideTable', '回溯天数': 1, '只起始日回溯': True, '只回溯非目标日': False, '只回溯时点': False}
数据 : 
            factor1  factor2
2020-01-04      3.0      4.0
2020-01-05      NaN      6.0
2020-01-06      5.0      7.0
2020-01-07      NaN      NaN
2020-01-08      8.0      9.0
2020-01-09      NaN      NaN


In [7]:
# WideTable 只回溯非目标日
Args = {
    "因子表类型": "WideTable", 
    "回溯天数": 1, # 0, 1, 2, np.inf
    "只起始日回溯": False, 
    "只回溯非目标日": True, 
    "只回溯时点": False
}
FT = SQLite3DB.getTable("test_WideTable1", args=Args)
DTs = [dt.datetime(2020,1,4)+dt.timedelta(i) for i in range(4)]+[dt.datetime(2020,1,9)]
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=["000001.SZ"], dts=DTs).iloc[:, :, 0])

参数 : 
{'因子表类型': 'WideTable', '回溯天数': 1, '只起始日回溯': False, '只回溯非目标日': True, '只回溯时点': False}
数据 : 
            factor1  factor2
2020-01-04      3.0      4.0
2020-01-05      NaN      6.0
2020-01-06      5.0      7.0
2020-01-07      NaN      NaN
2020-01-09      8.0      9.0


In [8]:
# WideTable 只回溯时点
SQLStr = "SELECT * FROM test_WideTable2"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020, 2, 1), dt.datetime(2020, 3, 1)]
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]

Args = {
    "因子表类型": "WideTable", 
    "回溯天数": 90, # 0, 1, 2, np.inf
    "只起始日回溯": False, 
    "只回溯非目标日": False, 
    "只回溯时点": False
}
FT = SQLite3DB.getTable("test_WideTable2", args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

Args = {
    "因子表类型": "WideTable", 
    "回溯天数": 90, # 0, 1, 2, np.inf
    "只起始日回溯": False, 
    "只回溯非目标日": False, 
    "只回溯时点": True
}
FT = SQLite3DB.getTable("test_WideTable2", args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

库原始数据 : 
     datetime       code  factor1
0  2020-01-31  000001.SZ      1.0
1  2020-01-31  000002.SZ      2.0
2  2020-01-31  000003.SZ      NaN
3  2020-02-28  000001.SZ      NaN
4  2020-02-28  000002.SZ      3.0
5  2020-02-28  000003.SZ      4.0
参数 : 
{'因子表类型': 'WideTable', '回溯天数': 90, '只起始日回溯': False, '只回溯非目标日': False, '只回溯时点': False}
数据 : 
            000001.SZ  000002.SZ  000003.SZ
2020-02-01        1.0        2.0        NaN
2020-03-01        1.0        3.0        4.0
参数 : 
{'因子表类型': 'WideTable', '回溯天数': 90, '只起始日回溯': False, '只回溯非目标日': False, '只回溯时点': True}
数据 : 
ID          000001.SZ  000002.SZ  000003.SZ
2020-02-01        1.0        2.0        NaN
2020-03-01        NaN        3.0        4.0


### 公告时点

数据库数据示例：
![Wide_Table_AnnDT_Demo](../images/Wide_Table_AnnDT_Demo.png)

In [9]:
# WideTable 公告时点
SQLStr = "SELECT * FROM test_WideTable3"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(11)]
IDs = ["000001.SZ"]

Args = {
    "因子表类型": "WideTable", 
    "公告时点字段": None
}
FT = SQLite3DB.getTable("test_WideTable3", args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=IDs, dts=DTs).iloc[:, :, 0])

Args = {
    "因子表类型": "WideTable", 
    "公告时点字段": "ann_dt"
}
FT = SQLite3DB.getTable("test_WideTable3", args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=IDs, dts=DTs).iloc[:, :, 0])

库原始数据 : 
     datetime       code      ann_dt   factor1   factor2
0  2020-01-01  000001.SZ  2020-01-01  0.421501  0.764511
1  2020-01-03  000001.SZ  2020-01-05  0.099590  0.181232
2  2020-01-05  000001.SZ  2020-01-07  0.981208  0.223434
3  2020-01-10  000001.SZ  2020-01-09  0.571260  0.194641
参数 : 
{'因子表类型': 'WideTable', '公告时点字段': None}
数据 : 
             factor1   factor2
2020-01-01  0.421501  0.764511
2020-01-02       NaN       NaN
2020-01-03  0.099590  0.181232
2020-01-04       NaN       NaN
2020-01-05  0.981208  0.223434
2020-01-06       NaN       NaN
2020-01-07       NaN       NaN
2020-01-08       NaN       NaN
2020-01-09       NaN       NaN
2020-01-10  0.571260  0.194641
2020-01-11       NaN       NaN
参数 : 
{'因子表类型': 'WideTable', '公告时点字段': 'ann_dt'}
数据 : 
             factor1   factor2
2020-01-01  0.421501  0.764511
2020-01-02       NaN       NaN
2020-01-03       NaN       NaN
2020-01-04       NaN       NaN
2020-01-05  0.099590  0.181232
2020-01-06       NaN       NaN
2020-01-07 

### 多重映射

数据库数据示例：
![Wide_Table_MultiMapping_Demo](../images/Wide_Table_MultiMapping_Demo.png)

In [10]:
# WideTable 多重映射
TargetTable = "test_WideTable4"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(3)]
IDs = ["000001.SZ", "000002.SZ"]

Args = {
    "因子表类型": "WideTable", 
    "多重映射": True
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print(f"factor1 数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])
print(f"factor2 数据 : ")
print(FT.readData(factor_names=["factor2"], ids=IDs, dts=DTs).iloc[0, :, :])

库原始数据 : 
     datetime       code  factor1  factor2
0  2020-01-01  000001.SZ      0.0   0.0779
1  2020-01-01  000001.SZ      1.0   0.4519
2  2020-01-01  000001.SZ      2.0   0.9420
3  2020-01-01  000002.SZ      0.0   0.7671
4  2020-01-01  000002.SZ      1.0   0.8653
5  2020-01-02  000001.SZ      0.0   0.9992
6  2020-01-02  000001.SZ      1.0   0.7077
7  2020-01-02  000002.SZ      0.0   0.4885
8  2020-01-02  000002.SZ      1.0   0.4779
9  2020-01-02  000002.SZ      2.0   0.8624
参数 : 
{'因子表类型': 'WideTable', '多重映射': True}
factor1 数据 : 
                  000001.SZ        000002.SZ
2020-01-01  [0.0, 1.0, 2.0]       [0.0, 1.0]
2020-01-02       [0.0, 1.0]  [0.0, 1.0, 2.0]
2020-01-03              NaN              NaN
factor2 数据 : 
                          000001.SZ                 000002.SZ
2020-01-01  [0.0779, 0.4519, 0.942]          [0.7671, 0.8653]
2020-01-02         [0.9992, 0.7077]  [0.4885, 0.4779, 0.8624]
2020-01-03                      NaN                       NaN


In [11]:
# WideTable 多重映射, 排序
TargetTable = "test_WideTable4"
DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(3)]
IDs = ["000001.SZ", "000002.SZ"]

Args = {
    "因子表类型": "WideTable", 
    "多重映射": True,
    "排序字段": [("factor1", "DESC")]
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print(f"factor1 数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])
print(f"factor2 数据 : ")
print(FT.readData(factor_names=["factor2"], ids=IDs, dts=DTs).iloc[0, :, :])

参数 : 
{'因子表类型': 'WideTable', '多重映射': True, '排序字段': [('factor1', 'DESC')]}
factor1 数据 : 
                  000001.SZ        000002.SZ
2020-01-01  [2.0, 1.0, 0.0]       [1.0, 0.0]
2020-01-02       [1.0, 0.0]  [2.0, 1.0, 0.0]
2020-01-03              NaN              NaN
factor2 数据 : 
                          000001.SZ                 000002.SZ
2020-01-01  [0.942, 0.4519, 0.0779]          [0.8653, 0.7671]
2020-01-02         [0.7077, 0.9992]  [0.8624, 0.4779, 0.4885]
2020-01-03                      NaN                       NaN


In [12]:
# WideTable 多重映射, 算子
TargetTable = "test_WideTable4"
DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(3)]
IDs = ["000001.SZ", "000002.SZ"]

def tmp_operator(x):
    return x.astype(np.float).mean()

Args = {
    "因子表类型": "WideTable", 
    "多重映射": True,
    "算子": tmp_operator,
    "算子数据类型": "double"
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print(f"factor1 数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])
print(f"factor2 数据 : ")
print(FT.readData(factor_names=["factor2"], ids=IDs, dts=DTs).iloc[0, :, :])

参数 : 
{'因子表类型': 'WideTable', '多重映射': True, '算子': <function tmp_operator at 0x0000026B899DAEE0>, '算子数据类型': 'double'}
factor1 数据 : 
            000001.SZ  000002.SZ
2020-01-01        1.0        0.5
2020-01-02        0.5        1.0
2020-01-03        NaN        NaN
factor2 数据 : 
            000001.SZ  000002.SZ
2020-01-01    0.49060     0.8162
2020-01-02    0.85345     0.6096
2020-01-03        NaN        NaN


In [13]:
# WideTable 多重映射, 算子, 附加字段
TargetTable = "test_WideTable4"
DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(3)]
IDs = ["000001.SZ", "000002.SZ"]

def tmp_operator(df):
    return df.astype(np.float).sum(axis=1).mean()

Args = {
    "因子表类型": "WideTable", 
    "多重映射": True,
    "算子": tmp_operator,
    "算子数据类型": "double",
    "附加字段": ["factor2"]
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print(f"factor1 数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

参数 : 
{'因子表类型': 'WideTable', '多重映射': True, '算子': <function tmp_operator at 0x0000026BE6158DC0>, '算子数据类型': 'double', '附加字段': ['factor2']}
factor1 数据 : 
            000001.SZ  000002.SZ
2020-01-01    1.49060     1.3162
2020-01-02    1.35345     1.6096
2020-01-03        NaN        NaN


### 回溯期

In [14]:
# WideTable 回溯期
TargetTable = "test_WideTable5"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020,2,28), dt.datetime(2020,3,31)]
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]

Args = {
    "因子表类型": "WideTable", 
    "回溯期数": None,
    "原始值回溯天数": 0
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

Args = {
    "因子表类型": "WideTable", 
    "回溯期数": 1,
    "原始值回溯天数": 0
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

Args = {
    "因子表类型": "WideTable", 
    "回溯期数": 1,
    "原始值回溯天数": 60
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

库原始数据 : 
     datetime       code  factor1  factor2
0  2020-01-31  000001.SZ      1.0   0.4758
1  2020-01-31  000002.SZ      2.0   0.9686
2  2020-02-28  000002.SZ      3.0   0.6583
3  2020-02-28  000003.SZ      4.0   0.8035
4  2020-03-31  000001.SZ      5.0   0.4362
5  2020-03-31  000002.SZ      6.0   0.7328
6  2020-03-31  000003.SZ      7.0   0.7527
参数 : 
{'因子表类型': 'WideTable', '回溯期数': None, '原始值回溯天数': 0}
数据 : 
            000001.SZ  000002.SZ  000003.SZ
2020-02-28        NaN        3.0        4.0
2020-03-31        5.0        6.0        7.0
参数 : 
{'因子表类型': 'WideTable', '回溯期数': 1, '原始值回溯天数': 0}
数据 : 
            000001.SZ  000002.SZ  000003.SZ
2020-02-28        NaN        NaN        NaN
2020-03-31        NaN        3.0        4.0
参数 : 
{'因子表类型': 'WideTable', '回溯期数': 1, '原始值回溯天数': 60}
数据 : 
            000001.SZ  000002.SZ  000003.SZ
2020-02-28        NaN        2.0        NaN
2020-03-31        1.0        3.0        4.0


## FeatureTable

继承自 WideTable, 参数集:
* 回溯天数: float, 默认值 inf, 缺失填充回溯的天数
* 目标时点: None(默认值) 或者 datetime, 截面所属的时点：
    + 时点字段为 None: 将库表中原始截面数据按照真实的时点序列进行填充得到最终数据
    + 时点字段不为 None: 目标时点如果为 None 则自动转换为时点字段的最大值，以目标时点形成的单时点序列调用 WideTable 提取原始数据的方法得到原始数据，最后以目标时点形成的单时点序列调用 WideTable 的转换方法得到因子数据并按照真实的时点序列进行填充得到最终数据

数据库数据示例：
![Feature_Table_Demo](../images/Feature_Table_Demo.png)

In [15]:
# FeatureTable 时点字段为 None
TargetTable = "test_FeatureTable1"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020, 1, 1), dt.datetime(2020, 1, 2), dt.datetime(2020, 1, 3)]

Args = {
    "因子表类型": "FeatureTable",
    "回溯天数": 0,
    "时点字段": None,
    "目标时点": None,
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
IDs = FT.getID()[:5]
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

库原始数据 : 
        code   factor1   factor2
0  000001.SZ  0.421501  0.764511
1  000002.SZ  0.099590  0.181232
2  000003.SZ  0.981208  0.223434
3  000004.SZ  0.571260  0.194641
4  000005.SZ  0.870928  0.910735
5  000006.SZ  0.183196  0.350026
6  000007.SZ  0.537513  0.056520
参数 : 
{'因子表类型': 'FeatureTable', '回溯天数': 0, '时点字段': None, '目标时点': None}
数据 : 
            000001.SZ  000002.SZ  000003.SZ  000004.SZ  000005.SZ
2020-01-01   0.421501    0.09959   0.981208    0.57126   0.870928
2020-01-02   0.421501    0.09959   0.981208    0.57126   0.870928
2020-01-03   0.421501    0.09959   0.981208    0.57126   0.870928


In [16]:
# FeatureTable 时点字段为不为 None
TargetTable = "test_FeatureTable2"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2021, 1, 1), dt.datetime(2021, 1, 2), dt.datetime(2021, 1, 3)]
IDs = [str(i).zfill(6)+".SZ" for i in range(1, 8)]

Args = {
    "因子表类型": "FeatureTable",
    "回溯天数": np.inf,# 0, 1, ..., np.inf
    "时点字段": "end_dt",
    "目标时点": None,# None, datetime
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1"], ids=IDs, dts=DTs).iloc[0, :, :])

库原始数据 : 
        code      end_dt   factor1   factor2
0  000001.SZ  2020-01-01  0.421501  0.764511
1  000002.SZ  2020-01-02  0.099590  0.181232
2  000003.SZ  2020-01-03  0.981208  0.223434
3  000004.SZ  2020-01-04  0.571260  0.194641
4  000005.SZ  2020-01-05  0.870928  0.910735
5  000006.SZ  2020-01-06  0.183196  0.350026
6  000007.SZ  2020-01-07  0.537513  0.056520
参数 : 
{'因子表类型': 'FeatureTable', '回溯天数': inf, '时点字段': 'end_dt', '目标时点': None}
数据 : 
            000001.SZ  000002.SZ  000003.SZ  000004.SZ  000005.SZ  000006.SZ  \
2021-01-01   0.421501    0.09959   0.981208    0.57126   0.870928   0.183196   
2021-01-02   0.421501    0.09959   0.981208    0.57126   0.870928   0.183196   
2021-01-03   0.421501    0.09959   0.981208    0.57126   0.870928   0.183196   

            000007.SZ  
2021-01-01   0.537513  
2021-01-02   0.537513  
2021-01-03   0.537513  


## TimeSeriesTable

继承自 SQL_Table, 参数集:
* 缺失填充相关参数:
    + 回溯天数: 参见 [WideTable](#WideTable)
    + 只起始日回溯: 参见 [WideTable](#WideTable)
    + 只回溯非目标日: 参见 [WideTable](#WideTable)
    + 只回溯时点: 参见 [WideTable](#WideTable)
* 公告时点字段: 参见 [WideTable](#WideTable)
* 忽略时间: 参见 [WideTable](#WideTable)
* 截止日期递增: 参见 [WideTable](#WideTable)
* 排序字段: 参见 [WideTable](#WideTable)
* 多重映射相关参数:
    + 多重映射: 参见 [WideTable](#WideTable)
    + 算子: 参见 [WideTable](#WideTable)
    + 算子数据类型: 参见 [WideTable](#WideTable)

当做 ID 字段为单一值的 WideTable 处理，每个时点所有 ID 填充同一个值

数据库数据示例：
![TimeSeries_Table_Demo](../images/TimeSeries_Table_Demo.png)

## NarrowTable

* 缺失填充相关参数:
    + 回溯天数: 参见 [WideTable](#WideTable)
    + 只起始日回溯: 参见 [WideTable](#WideTable)
    + 只回溯非目标日: 参见 [WideTable](#WideTable)
    + 只回溯时点: 参见 [WideTable](#WideTable)
* 因子名字段: str, 默认值自动判断, 指示因子名称的字段
* 因子值字段: str, 默认值自动判断, 指示因子取值的字段
* 多重映射相关参数:
    + 多重映射: 参见 [WideTable](#WideTable)
    + 算子: 参见 [WideTable](#WideTable)
    + 算子数据类型: 参见 [WideTable](#WideTable)

数据库数据示例：
![Narrow_Table_Demo](../images/Narrow_Table_Demo.png)

In [3]:
# NarrowTable
TargetTable = "test_NarrowTable1"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2020, 1, 1)+dt.timedelta(i) for i in range(4)]
IDs = ["000001.SZ"]

Args = {
    "因子表类型": "NarrowTable",
    "因子名字段": "name",
    "因子值字段": "value",
    "回溯天数": 0,
    "多重映射": False
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=IDs, dts=DTs).iloc[:, :, 0])

库原始数据 : 
     datetime       code     name     value
0  2020-01-01  000001.SZ  factor1  0.764511
1  2020-01-02  000001.SZ  factor1  0.181232
2  2020-01-03  000001.SZ  factor1  0.223434
3  2020-01-01  000001.SZ  factor2  0.194641
4  2020-01-02  000001.SZ  factor2  0.910735
参数 : 
{'因子表类型': 'NarrowTable', '因子名字段': 'name', '因子值字段': 'value', '回溯天数': 0, '多重映射': False}
数据 : 
             factor1   factor2
2020-01-01  0.764511  0.194641
2020-01-02  0.181232  0.910735
2020-01-03  0.223434       NaN
2020-01-04       NaN       NaN


## MappingTable

继承自 SQL_Table, 参数集:
* 只填起始日: bool, 默认值 False, 是否将数据只填充在起始时点
* 多重映射: bool, 默认值 False, 是否为高维数据, 即起始时点和 ID 两个维度无法唯一索引单个数据, 默认形成的数据在单个时点单个 ID 处以 list 形式表达
* 结束时点字段: str, 默认值自动判断, 用以指示结束填充的时点字段
* 包含结束时点: bool, 默认值 True, 结束时点处是否填充数据

数据库数据示例：
![Mapping_Table_Demo](../images/Mapping_Table_Demo.png)

In [4]:
# MappingTable
TargetTable = "test_MappingTable1"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2019, 12, 31)+dt.timedelta(i) for i in range(12)]
IDs = ["000001.SZ"]

Args = {
    "因子表类型": "MappingTable",
    "时点字段": "datetime",
    "结束时点字段": "end_dt",
    "只填起始日": True,
    "包含结束时点": True,
    "多重映射": False
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=IDs, dts=DTs).iloc[:, :, 0])

库原始数据 : 
        code    datetime      end_dt  factor1   factor2
0  000001.SZ  2020-01-01  2020-01-02      1.0  0.764511
1  000001.SZ  2020-01-03  2020-01-05      1.0  0.181232
2  000001.SZ  2020-01-07  2020-01-10      1.0  0.223434
3  000001.SZ  2020-01-09  2020-01-10      2.0  0.223434
参数 : 
{'因子表类型': 'MappingTable', '时点字段': 'datetime', '结束时点字段': 'end_dt', '只填起始日': True, '包含结束时点': True, '多重映射': False}
数据 : 
           factor1   factor2
2019-12-31     NaN       NaN
2020-01-01     1.0  0.764511
2020-01-02     NaN       NaN
2020-01-03     1.0  0.181232
2020-01-04     NaN       NaN
2020-01-05     NaN       NaN
2020-01-06     NaN       NaN
2020-01-07     1.0  0.223434
2020-01-08     NaN       NaN
2020-01-09     2.0  0.223434
2020-01-10     NaN       NaN
2020-01-11     NaN       NaN


## ConstituentTable

参数集:
* 类别字段: str, 默认值自动判断, 作为因子名称的字段
* 结束时点字段: str, 默认值自动判断, 用以指示调出成份的时点字段
* 包含结束时点: bool, 默认值 False, 结束时点处是否包含在成份中

![Constituent_Table_Demo](../images/Constituent_Table_Demo.png)

In [5]:
# ConstituentTable
TargetTable = "test_ConstituentTable1"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2019, 12, 31)+dt.timedelta(i) for i in range(12)]
IDs = ["000001.SZ", "000002.SZ", "000003.SZ", "600000.SH"]

Args = {
    "因子表类型": "ConstituentTable",
    "时点字段": "datetime",
    "类别字段": "group_code",
    "结束时点字段": "end_dt",
    "包含结束时点": False
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["000300.SH"], ids=IDs, dts=DTs).iloc[0, :, :])

库原始数据 : 
  group_code       code    datetime      end_dt  current_status
0  000300.SH  000001.SZ  2020-01-01  2020-02-28             NaN
1  000300.SH  000002.SZ  2020-01-01  2020-02-28             NaN
2  000300.SH  600000.SH  2020-01-01        None             1.0
3  000905.SH  000001.SZ  2020-01-02  2020-03-31             NaN
4  000905.SH  000002.SZ  2020-01-02        None             1.0
参数 : 
{'因子表类型': 'ConstituentTable', '时点字段': 'datetime', '类别字段': 'group_code', '结束时点字段': 'end_dt', '包含结束时点': False}
数据 : 
            000001.SZ  000002.SZ  000003.SZ  600000.SH
2019-12-31          0          0          0          0
2020-01-01          1          1          0          1
2020-01-02          1          1          0          1
2020-01-03          1          1          0          1
2020-01-04          1          1          0          1
2020-01-05          1          1          0          1
2020-01-06          1          1          0          1
2020-01-07          1          1          0   

## FinancialTable

参数集:
* 报告期: str, 可选 "所有"(默认值), "定期报告", "年报", "中报", "一季报", "三季报", 指定原始数据中保留的报告期报告
* 计算方法: str, 可选 "最新"(默认值), "单季度", "TTM":
    - 最新: 以当前时点能得到的(公告时点在当前时点之前)指定报告期的财务报告的数据值作为当前时点的因子值.
    - 单季度: 以当前时点能得到的(公告时点在当前时点之前)指定报告期的财务报告的数据值计算出的单季度数据作为当前时点的因子值. 比如, 当前时点是 2010 年 8 月 20 日, 指定的报告期是所有, 要计算最新单季度的净利润, 如果某公司在这天以前已经披露了 2010 年中报, 则用 2010 年中报的净利润减去 2010 年一季报的净利润得到二季度净利润作为当前最新单季度净利润因子值, 否则以 2010 年一季报的净利润值直接作为因子值.
    - TTM: 以当前时点能得到的(公告时点在当前时点之前)指定报告期的财务报告的数据值计算出的滚动四季度数据作为当前时点的因子值. 比如, 当前时点是 2010 年 8 月 20 日, 指定的报告期是所有, 要计算最新 TTM 的净利润, 如果某公司在这天以前已经披露了 2010 年中报, 则用 2010 年中报的净利润加上 2009 年年报的净利润再减去 2009 年中报的净利润值得到过去滚动四季度的净利润作为当前最新 TTM 净利润因子值, 否则以 2010 年一季报的净利润加上 2009 年年报的净利润再减去 2009 年一季报的净利润的值直接作为因子值.
* 回溯年数: int, 默认值 0. 给定回溯年数 n, 搜索当前时点能得到的(公告时点在当前时点之前)指定报告期的最新财务报表的报告期, 将该报告期减去 n年后得到的报告期对应的财务报表数据值作为当前时点的因子值. 比如, 给定回溯年数为 1, 当前时点是 2010 年 8 月 20 日, 如果某公司在这天以前已经披露了中报, 则以 2009 年的中报值作为当前因子值, 否则以 2009 年一季报的值作为因子值. 回溯 n 年最新年报、回溯 n 年最新单季度以及回溯 n 年 TTM 的变换方式可以类推.
* 回溯期数: int, 默认值 0. 给定回溯期数 n, 搜索当前时点能得到的(公告时点在当前时点之前)指定报告期的最新财务报表的报告期, 将该报告期减去 n期后得到的报告期对应的财务报表数据值作为当前时点的因子值. 比如, 给定回溯期数为 1, 当前时点是 2010 年 8 月 20 日, 如果某公司在这天以前已经披露了中报, 则以 2010 年的一季报的值作为当前因子值, 否则以 2009 年年报的值作为因子值. 回溯 n 期最新年报、回溯 n 期最新单季度以及回溯 n 期 TTM 的变换方式可以类推.
* 公告时点字段: str, 默认值自动判断, 指定报告的披露时点
* 忽略缺失: bool, 默认值 True
* 忽略非季末报告: bool, 默认值 False
* 调整类型字段: None 或者 str, 默认值自动判断
* 调整类型: str

![Financial_Table_Demo](../images/Financial_Table_Demo.png)

In [6]:
# FinancialTable
TargetTable = "test_FinancialTable1"
SQLStr = f"SELECT * FROM {TargetTable}"
print("库原始数据 : ")
print(pd.read_sql_query(SQLStr, SQLite3DB.Connection))

DTs = [dt.datetime(2022,1,17), dt.datetime(2022,1,18)]
IDs = ["000001.SZ"]

Args = {
    "因子表类型": "FinancialTable",
    "时点字段": "report_date",
    "公告时点字段": "ann_dt",
    "报告期": "年报",
    "计算方法": "最新",
    "回溯年数": 1,
    "回溯期数": 0,
}
FT = SQLite3DB.getTable(TargetTable, args=Args)
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["factor1", "factor2"], ids=IDs, dts=DTs).iloc[:, :, 0])

库原始数据 : 
         code report_date      ann_dt  adj_type  factor1   factor2
0   000001.SZ  2019-03-31  2019-04-16         2      1.0  0.243601
1   000001.SZ  2019-06-30  2019-07-28         2      2.0  0.228097
2   000001.SZ  2019-09-30  2020-10-23         2      4.0  0.982368
3   000001.SZ  2019-12-31  2021-03-22         2      7.0  0.561210
4   000001.SZ  2020-03-31  2020-04-15         2     11.0  0.567778
5   000001.SZ  2020-06-30  2020-07-20         2     16.0  0.789474
6   000001.SZ  2020-09-30  2020-10-15         2     22.0  0.024179
7   000001.SZ  2019-12-31  2021-03-23         1     29.0  0.323520
8   000001.SZ  2020-12-31  2021-03-23         2     37.0  0.790493
9   000001.SZ  2020-03-31  2021-04-21         1     46.0  0.722441
10  000001.SZ  2021-03-31  2021-04-21         2     56.0  0.298621
11  000001.SZ  2021-06-30  2021-07-21         2     67.0  0.508704
12  000001.SZ  2021-09-30  2021-10-30         2     79.0  0.799892
13  000001.SZ  2020-12-31  2022-01-18         1     9